In [3]:
import pandas as pd
import numpy as np
# import pm4py
import copy
import random
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
#1.加载模型

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error,  mean_squared_error,r2_score,median_absolute_error,explained_variance_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
# from xgboost.sklearn import XGBRegressor
# from lightgbm.sklearn import LGBMRegressor
# #模型实例化
# model = LinearRegression(normalize=True)#其他模型类似
# #向模型中填充数据
# model = model.fit(train_X, train_Y)
#模型预测

In [4]:
data=pd.read_csv('shuju_model1.csv')

In [5]:
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,label,fitness
0,D,B,D,E,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.800000
1,A,C,D,A,C,F,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.833333
2,A,C,B,H,F,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.818182
3,A,C,C,C,C,D,G,F,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.785714
4,A,C,G,D,A,F,I,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.916667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6671,A,C,G,G,G,G,G,G,D,G,G,F,I,NaN,NaN,NaN,NaN,NaN,1,1.000000
6672,A,C,G,G,G,G,G,G,G,H,D,F,I,NaN,NaN,NaN,NaN,NaN,1,1.000000
6673,A,C,G,G,G,G,G,G,G,D,H,F,I,NaN,NaN,NaN,NaN,NaN,1,1.000000
6674,A,C,G,G,G,G,G,G,G,D,G,F,I,NaN,NaN,NaN,NaN,NaN,1,1.000000


# 打乱不同标签的数据

In [6]:
new_labels=data["label"]
data.drop(columns=['label'],inplace=True)

In [7]:
new_fitness=data["fitness"]
data.drop(columns=["fitness"],inplace=True)

In [8]:
dataset=data

In [9]:
dataset=dataset.replace(['A','B','C','D','E','F','G','H','I'],[0,1,2,3,4,5,6,7,8])

In [10]:
dataset=dataset.fillna(9)

In [11]:
# 3.1 数据类型转换
dataset_tensor = torch.Tensor(dataset.values)

# 对datafram 进行独热编码

In [12]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
num_trace=data.shape[0]
len_trace=dataset.shape[1]
num_unique=10

dd=torch.zeros(num_trace, len_trace,num_unique, dtype=torch.long)
#dd(num_trace,len_trace,num_unique)
def one_hot(x, n_class, dtype=torch.float32): 
    # X shape: (batch), output shape: (batch, n_class)
    x = x.long()
    res = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    res.scatter_(1, x.view(-1, 1), 1)
    return res
def to_onehot(X, n_class):  
    # X shape: (batch, seq_len), output: seq_len elements of (batch, n_class)
    for i in range(X.shape[0]):
        dd[i,:,:]=one_hot(X[i,:], n_class) 
    return dd



In [13]:
ceshi=to_onehot(dataset_tensor,num_unique)

# 增加一个batch_size维度

In [14]:
ceshi=ceshi.view(ceshi.shape[0],-1)

In [15]:
ceshi=ceshi.numpy()

In [16]:
ceshi=pd.DataFrame(ceshi)

In [17]:
new_fitness

0       0.800000
1       0.833333
2       0.818182
3       0.785714
4       0.916667
          ...   
6671    1.000000
6672    1.000000
6673    1.000000
6674    1.000000
6675    1.000000
Name: fitness, Length: 6676, dtype: float64

In [18]:
ceshi

,0,1,2,3,4,5,6,7,8,9,...,170,171,172,173,174,175,176,177,178,179
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6671,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6672,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6673,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6674,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# 使用pac降维

In [19]:
from sklearn.decomposition import PCA
pca = PCA(n_components=180)
pca.fit(ceshi)
ceshi=pca.transform(ceshi)

In [20]:
# x_train, x_test, y_train, y_test = train_test_split(ceshi, new_fitness, random_state=0, train_size=0.8)

In [21]:
from sklearn.model_selection import KFold

In [22]:
kf = KFold(n_splits=10,shuffle=True,random_state=1)

In [23]:
new_fitness

0       0.800000
1       0.833333
2       0.818182
3       0.785714
4       0.916667
          ...   
6671    1.000000
6672    1.000000
6673    1.000000
6674    1.000000
6675    1.000000
Name: fitness, Length: 6676, dtype: float64

In [24]:
list_a=[]
for train_index, test_index in kf.split(ceshi):
#     print('train_index', train_index, 'test_index', test_index)
    x_train, y_train = ceshi[train_index], new_fitness[train_index]
    x_test, y_test = ceshi[test_index], new_fitness[test_index]
    models = [DecisionTreeRegressor(random_state=1)]
    asses_method=['mean_absolute_error','mean_squared_error','r2_score']
    #将各模型结果保存在字典中
    result =[[]*i for i in range(0,len(models))]
    i=-1
    for model in models:
        i=i+1
#         print(model)
        for ass_method in asses_method:
                model.fit(x_train, y_train)
                score = eval(ass_method)(y_test, model.predict(x_test))
#                 print(score)
                result[i].append(score)
#     print(result)
    list_a.append(result)
#     ass_data = pd.DataFrame(result) 
#     ass_data.rename(columns={0:"mean_absolute_error",1:"mean_squared_error"},inplace=True)
#     ass_data.index=["DecisionTreeRegressor"]
#     ass_data
cca=np.zeros(3)
for i in range(0,len(list_a)):
    cca= np.array(list_a[i][0])+cca
print(cca/10)

[4.91141267e-03 4.50740449e-04 8.83672165e-01]


In [35]:
from sklearn.cluster import AgglomerativeClustering
from collections import Counter
km = AgglomerativeClustering(n_clusters=5).fit(ceshi)

In [26]:
# from sklearn.cluster import KMeans
# from collections import Counter
# km = KMeans(n_clusters=3).fit(ceshi)

In [27]:
# from sklearn.cluster import Birch
# from collections import Counter
# km = Birch(n_clusters=3).fit(ceshi)

In [39]:
km.labels_

array([0, 0, 0, ..., 3, 3, 3], dtype=int64)

In [28]:
count = Counter(km.labels_)

In [29]:
list_temp=[]
for i in range(3):
    list_=ceshi[np.where(km.labels_==i)[0]]
    label_=new_fitness[np.where(km.labels_==i)[0]]
    kf = KFold(n_splits=10,shuffle=True,random_state=1)
    list_a=[]
    label_.reset_index(drop=True,inplace=True)
    for train_index, test_index in kf.split(list_):
        x_train, y_train = list_[train_index], label_[train_index]
        x_test, y_test = list_[test_index], label_[test_index]
        models = [DecisionTreeRegressor(random_state=1)]
        asses_method=['mean_absolute_error','mean_squared_error','r2_score']
        #将各模型结果保存在字典中
        result =[[]*i for i in range(0,len(models))]
        i=-1
        for model in models:
            i=i+1
    #         print(model)
            for ass_method in asses_method:
                model.fit(x_train, y_train)
                score = eval(ass_method)(y_test, model.predict(x_test))
                print(ass_method)
                print(score)
                result[i].append(score)
        list_a.append(result)
#         print(result)
    ccb=np.zeros(3)
    for i in range(0,len(list_a)):
        ccb= np.array(list_a[i][0])+ccb
#     print(ccb/5)
    list_temp.append((ccb/10).tolist())
    #     break

mean_absolute_error
0.004483187842818478
mean_squared_error
0.0002723152648599124
r2_score
0.9076482503432031
mean_absolute_error
0.004767975989159929
mean_squared_error
0.00027618521578556154
r2_score
0.8940501194242959
mean_absolute_error
0.006737701642276448
mean_squared_error
0.0004934632121932459
r2_score
0.8256438410564404
mean_absolute_error
0.004219439151761542
mean_squared_error
0.00026014608296767093
r2_score
0.9006620100909095
mean_absolute_error
0.004622122745257491
mean_squared_error
0.0002983766951809456
r2_score
0.8795037540126247
mean_absolute_error
0.004957492758152199
mean_squared_error
0.0002847531178610426
r2_score
0.8975270351354872
mean_absolute_error
0.004413632915760901
mean_squared_error
0.00022355689624869435
r2_score
0.9031808032735139
mean_absolute_error
0.004463282978260901
mean_squared_error
0.0003376550730768384
r2_score
0.8677038760746552
mean_absolute_error
0.004616396187500038
mean_squared_error
0.00033576846339570794
r2_score
0.8732400572377728
mean_a

In [30]:
dict1=dict(count)

In [31]:
list_temp

[[0.004848178952616534, 0.00032089373802717734, 0.8768226422608718],
 [0.006768705612991958, 0.000765897516981533, 0.8926533140872396],
 [0.0018605065007516654, 0.00014410866455076326, 0.8993295311855001]]

In [32]:
cc0=pd.DataFrame(list_temp[0])
cc1=pd.DataFrame(list_temp[1])
cc2=pd.DataFrame(list_temp[2])
# cc3=pd.DataFrame(list_temp[3])

In [33]:
cc0*dict1[0]/sum(dict1.values())+cc1*dict1[1]/sum(dict1.values())+cc2*dict1[2]/sum(dict1.values())

,0
0,0.004714
1,0.000394
2,0.885268


In [34]:
print(cca/10)

[4.91141267e-03 4.50740449e-04 8.83672165e-01]


# ----------------------------------------------------------

# -------------------------------------------------------

In [33]:
x_train, x_test, y_train, y_test = train_test_split(ceshi, new_fitness, random_state=0, train_size=0.8)

In [34]:
models = [DecisionTreeRegressor(),
          RandomForestRegressor(),
          GradientBoostingRegressor(),
          MLPRegressor(solver='lbfgs', max_iter=1000),
          XGBRegressor(n_estimators = 1000, objective='reg:squarederror'), 
          LGBMRegressor(n_estimators = 1000)]
asses_method=['mean_absolute_error','mean_squared_error','r2_score','median_absolute_error','explained_variance_score']
#将各模型结果保存在字典中
result =[[]*i for i in range(0,len(models))]
i=-1
for model in models:
    i=i+1
    print(model)
    for ass_method in asses_method:

            model.fit(x_train, y_train)
            score = eval(ass_method)(y_test, model.predict(x_test))
            print(score)
            result[i].append(score)

ass_data = pd.DataFrame(result) 
ass_data.rename(columns={0:"mean_absolute_error",1:"mean_squared_error",2:"r2_score",3:"median_absolute_error",4:"explained_variance_score"},inplace=True)
ass_data.index=["DecisionTreeRegressor","RandomForestRegressor","GradientBoostingRegressor","MLPRegressor","XGBRegressor","LGBMRegressor"]

DecisionTreeRegressor()
0.005974169922155733
0.0004542145912773335
0.8430988027575677
0.0
0.883284384650679
RandomForestRegressor()
0.0049756608918788275
0.0003339611382987627
0.9131499148340271
0.0
0.9115342051839571
GradientBoostingRegressor()
0.008525327314472727
0.0004270918237877772
0.8849738563489744
0.0008171134487385823
0.8849023374273866
MLPRegressor(max_iter=1000, solver='lbfgs')
0.015042542622289934
0.0007394628667753059
0.818760999492907
0.01017412438705334
0.792335146820721
XGBRegressor(base_score=None, booster=None, colsample_bylevel=None,
             colsample_bynode=None, colsample_bytree=None, gamma=None,
             gpu_id=None, importance_type='gain', interaction_constraints=None,
             learning_rate=None, max_delta_step=None, max_depth=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None,
             random_state=None, reg_alpha=None, reg_lambda=None,
         

In [35]:
ass_data

,mean_absolute_error,mean_squared_error,r2_score,median_absolute_error,explained_variance_score
DecisionTreeRegressor,0.005974,0.000454,0.843099,0.000000,0.883284
RandomForestRegressor,0.004976,0.000334,0.913150,0.000000,0.911534
GradientBoostingRegressor,0.008525,0.000427,0.884974,0.000817,0.884902
MLPRegressor,0.015043,0.000739,0.818761,0.010174,0.792335
XGBRegressor,0.005321,0.000324,0.912841,0.000251,0.913085
LGBMRegressor,0.004966,0.000293,0.921216,0.000297,0.921369


# claster

# KMeans划分聚类

In [18]:
from collections import Counter

In [19]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3).fit(ceshi)

In [20]:
count = Counter(km.labels_)

In [21]:
count

Counter({0: 1646, 2: 3103, 1: 1927})

# Birch聚类

In [22]:
# from sklearn.cluster import Birch
# gas= Birch(n_clusters=3).fit(ceshi)
# count = Counter(gas.labels_)
# count

# MiniBatchKMeans

In [23]:
# from sklearn.cluster import  MiniBatchKMeans
# clustering = MiniBatchKMeans(n_clusters=3).fit(ceshi)

In [24]:
# count = Counter(clustering.labels_)
# count

In [25]:
list_=ceshi[np.where(km.labels_==1)[0]]

In [37]:
list_temp=[]
for i in range(3):
    list_=ceshi[np.where(km.labels_==i)[0]]
    label_=new_fitness[np.where(km.labels_==i)[0]]
    x_train, x_test, y_train, y_test = train_test_split(list_,label_, random_state=0, train_size=0.8)
    models = [DecisionTreeRegressor(),
          RandomForestRegressor(),
          GradientBoostingRegressor(),
          MLPRegressor(solver='lbfgs', max_iter=1000),
          XGBRegressor(n_estimators = 1000, objective='reg:squarederror'), 
          LGBMRegressor(n_estimators = 1000)]
    asses_method=['mean_absolute_error','mean_squared_error','r2_score']
    #将各模型结果保存在字典中
    result =[[]*i for i in range(0,len(models))]
    i=-1
    for model in models:
        i=i+1
#         print(model)
        for ass_method in asses_method:
                model.fit(x_train, y_train)
                score = eval(ass_method)(y_test, model.predict(x_test))
#                 print(score)
                result[i].append(score)
#     print(result)
#     ass_data = pd.DataFrame(result)
#     ass_data.rename(columns={0:"mean_absolute_error",1:"mean_squared_error",2:"r2_score"},inplace=True)
#     ass_data.index=["DecisionTreeRegressor","RandomForestRegressor","GradientBoostingRegressor","MLPRegressor","XGBRegressor","LGBMRegressor"]
#     data_test=ass_data.copy()
#     data_test=data_test+ass_data
#     print(ass_data)
    list_temp.append(result)
#     break

In [38]:
dict1={0: 1645, 1: 3674, 2: 1357}

In [39]:
cc0=pd.DataFrame(list_temp[0])
cc1=pd.DataFrame(list_temp[1])
cc2=pd.DataFrame(list_temp[2])
# cc3=pd.DataFrame(list_temp[3])

In [40]:
cc0*dict1[0]/sum(dict1.values())+cc1*dict1[1]/sum(dict1.values())+cc2*dict1[2]/sum(dict1.values())

,0,1,2
0,0.007414,0.000588,0.835786
1,0.007264,0.000361,0.909194
2,0.009139,0.000416,0.898276
3,0.017504,0.000686,0.789348
4,0.007535,0.000366,0.909042
5,0.007467,0.000350,0.915426


In [36]:
ass_data

,mean_absolute_error,mean_squared_error,r2_score,median_absolute_error,explained_variance_score
DecisionTreeRegressor,0.005974,0.000454,0.843099,0.000000,0.883284
RandomForestRegressor,0.004976,0.000334,0.913150,0.000000,0.911534
GradientBoostingRegressor,0.008525,0.000427,0.884974,0.000817,0.884902
MLPRegressor,0.015043,0.000739,0.818761,0.010174,0.792335
XGBRegressor,0.005321,0.000324,0.912841,0.000251,0.913085
LGBMRegressor,0.004966,0.000293,0.921216,0.000297,0.921369
